# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [ ]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [ ]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]


We use integer identifiers for the nodes for efficiency. Note that, unlike this example, in a real web graph, not every page will have in-links, nor will every page have out-links.

### Entropy
Entropy is a measure of the uncertainty in a dataset. In the context of data compression, it quantifies the limit to which a dataset can be compressed. Mathematically, the entropy \( H(X) \) is defined as:

$$H(X) = -\sum_{i=1}^{n} p(x_i) \log_2 p(x_i)$$

### Perplexity
Perplexity is a metric that quantifies how well a probability distribution predicts a sample, and it is directly related to entropy. In this PageRank assignment, we define perplexity as $2^{H(PR)}$, where $H(PR)$ is the entropy of the PageRank distribution.

### Why It's Important
Calculating the perplexity at each iteration helps us understand to what extent the PageRank distribution is nearing a steady state. In other words, if the perplexity is changing very little across iterations, then we can assume that the PageRank distribution has stabilized, and the iterations can be stopped.

### Threshold Selection
Regarding the threshold, a common practice is to set a very small positive number (e.g., $1 	imes 10^{-5}$). If the change in perplexity across consecutive iterations is smaller than this threshold, then the iterations can be stopped.

### Maximum Perplexity
The maximum perplexity of a PageRank distribution will be equal to the number of nodes in the graph. This is because when all nodes have equal PageRank values, the entropy will reach its maximum, thus maximizing the perplexity.



## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [ ]:

# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.

def page_rank_fixed_iter(nodes, edges, iterations=10):
    
    P = set()
    S = set()
    M = dict()
    L = dict()
    N = len(nodes)
    d = 0.85

    page_rank = dict()


    for node in nodes:
        P.add(node[0])

    S = P.copy()

    for edge in edges:

        if edge[0] in S:
            
            S.remove(edge[0])
        
        if edge[1] in M:

            M[edge[1]].append(edge[0])
        
        else:

            M[edge[1]] = [edge[0]]
        

        if edge[0] in L:

            L[edge[0]].append(edge[1])
        
        else:

            L[edge[0]] = [edge[1]]

    for page in P:

        page_rank[page] = 1 / len(P)

    i = 0

    while i < iterations:

        sinkPR = 0

        for page in S:

            sinkPR += page_rank[page]
        
        for page in P:

            new_page_rank = (1- d) / N
            new_page_rank += d * (sinkPR/N)

            for neighbor in M[page]:

                new_page_rank += d * page_rank[neighbor] / len(L[neighbor])
            
            page_rank[page] = new_page_rank
        
        i += 1
    
    
    return page_rank

print(page_rank_fixed_iter(small_nodes, small_edges, 1))
print()

print(page_rank_fixed_iter(small_nodes, small_edges, 10))
print()

print(page_rank_fixed_iter(small_nodes, small_edges, 100))


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [ ]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.

!wsl wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!wsl gzip -df vertices-edu.txt.gz
!wsl wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!wsl gzip -df edges-edu.txt.gz


There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [ ]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.

nodes = []
edges = []

with open("vertices-edu.txt", "r") as file:

    for line in file:

        content = line.strip().split()
        content[0] = int(content[0])
        nodes.append(tuple(content))

with open("edges-edu.txt", "r") as file:

    for line in file:

        content = line.strip().split()
        content[0] = int(content[0])
        content[1] = int(content[1])
        edges.append(tuple(content))

print(nodes[:10])
print(edges[:10])

print("Nodes length is " + str(len(nodes)))
print("Edges length is " + str(len(edges)))

         

Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall that we defined entropy when talking about data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [ ]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

import math

def page_rank(nodes, edges, threshold=1):
    
    P = set()
    S = set()
    M = dict()
    L = dict()
    N = len(nodes)
    d = 0.85

    print(N)

    page_rank = dict()


    for node in nodes:
        
        P.add(node[0])

    S = P.copy()

    for edge in edges:

        if edge[0] in S:
            
            S.remove(edge[0])
        
        if edge[1] in M:

            M[edge[1]].add(edge[0])
        
        else:

            M[edge[1]] = {edge[0]}
        

        if edge[0] in L:

            L[edge[0]].add(edge[1])
        
        else:

            L[edge[0]] = {edge[1]}
    

    for page in P:

        page_rank[page] = 1 / len(P)

    perplexity = N
    change_in_perplexity = float('inf')

    temp_page_ranks = dict()

    while change_in_perplexity > threshold:

        shannon_entropy = 0

        sinkPR = 0

        for page in S:
            
            sinkPR += page_rank[page]
        
        for page in P:

            new_page_rank = (1- d) / N
            new_page_rank += d * (sinkPR/N)

            if page in M:

                for neighbor in M[page]:

                    new_page_rank += d * page_rank[neighbor] / len(L[neighbor])
                
                temp_page_ranks[page] = new_page_rank

                shannon_entropy += temp_page_ranks[page] * math.log(1/temp_page_ranks[page], 2) 
        
        for page in temp_page_ranks.keys():
            
            page_rank[page] = temp_page_ranks[page]
        
        shannon_entropy *= -1
        
        previous_perplexity = perplexity
        print("Previous Perplexity is " + str(previous_perplexity))

        updated_perplexity = math.pow(2, shannon_entropy)
        print("Updated Perplexity is " + str(updated_perplexity))

        change_in_perplexity = abs(previous_perplexity - updated_perplexity)

        perplexity = updated_perplexity

        print("Change in perplexity is " + str(change_in_perplexity))
    
    return page_rank

PR = page_rank(nodes, edges, 1)


## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [ ]:
# TODO: List the document ID, domain name, and in-link count of the 60 websites with the highest in-link count

M = dict()

for edge in edges:

   if edge[1] in M:

      M[edge[1]].add(edge[0])
   
   else:

      M[edge[1]] = {edge[0]}

sorted_dict = dict(sorted(M.items(), key=lambda item : len(item[1]), reverse=True))

count = 0

for key, value in sorted_dict.items():
   
   if count >= 60:
      break
   
   print(f"Document ID : {key}, Domain Name : {len(sorted_dict[key])}, In-Link Count : {len(value)}")

   count += 1

print("Count is " + str(count))
   



Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [ ]:
# TODO: List the document ID, domain name, and PageRank of the 60 websites with the highest PageRank.

sorted_page_ranks = dict(sorted(PR.items(), key=lambda item : item[1], reverse=True))

count = 0

for key, value in sorted_page_ranks.items():
   
   if count >= 60:
      break
   
   print(f"Document ID : {key}, Domain Name : {nodes[key]}, Page Rank : {value}")

   count += 1

print("Count is " + str(count))


Finally, compute some summary statistics on this dataset.

In [ ]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);

source_node_count = 0

for node in nodes:

    if node[0] not in M:

        source_node_count += 1

result = source_node_count / len(nodes)

print(result)
